In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk   
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
from nltk import PorterStemmer, WordNetLemmatizer 

import string
import collections

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/yelp_cleaned.csv', encoding='latin-1', index_col=0)

In [16]:
data.corr()

,stars,cool,useful,funny,text_len
stars,1.000000,0.052555,-0.023479,-0.061306,-0.092193
cool,0.052555,1.000000,0.887102,0.764342,0.237975
useful,-0.023479,0.887102,1.000000,0.723406,0.290306
funny,-0.061306,0.764342,0.723406,1.000000,0.242552
text_len,-0.092193,0.237975,0.290306,0.242552,1.000000


In [ ]:
pd.DataFrame(x_train).iloc[1, 0]  

In [455]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Z=data.drop('textc', axis=1)
X=data['textc'] #will add other features back later on for another analysis;   

rs=9876
x_train, x_test, ztrain, ztest = train_test_split(X, Z, test_size=0.3, random_state=rs) 


#TfidfVectorizer
tfidfv=TfidfVectorizer(min_df=2, ngram_range=(1, 1), stop_words='english', max_features=10000, strip_accents='unicode', 
                           norm='l2')

xtrain=tfidfv.fit_transform(x_train.values.astype('U')).todense()
xtest=tfidfv.transform(x_test.values.astype('U')).todense()
col = ['feat_'+i for i in tfidfv.get_feature_names()]  #for TfidfVectorizer


_xtrain = pd.DataFrame(xtrain, columns=col)  #feature names added
_xtest = pd.DataFrame(xtest, columns=col)  #row index from 0 to 6999 (number of reviews);  

temptrain=ztrain.loc[:, ['cool', 'useful', 'funny', 'text_len', 'sentiment']].reset_index(drop=True)  #remove 'sentiment' needed;
trainfull=pd.concat([temptrain, _xtrain], axis=1)

temptest=ztest.loc[:, ['cool', 'useful', 'funny', 'text_len', 'sentiment']].reset_index(drop=True) #remove 'sentiment' needed;
testfull=pd.concat([temptest, _xtest], axis=1)

ytrain=ztrain['stars']
ytest=ztest['stars']

In [506]:
from sklearn.ensemble import RandomForestRegressor

result=pd.DataFrame()
rs=300
global result

'''
model=RandomForestRegressor(random_state=rs)
cname='RandomForestRegressor'

model=XGBClassifier(random_state=rs)
cname='XGBoost'
'''
model=LogisticRegression(random_state=rs)
cname='LogisticRegression'


model.fit(trainfull, ytrain)
pred=model.predict(testfull)
tpred=model.predict(trainfull)
    

#RFRegressor has not attributes of predict_proba; 
predproba=pd.DataFrame(model.predict_proba(testfull))
tpredproba=pd.DataFrame(model.predict_proba(trainfull))
#merge with test and train labels; 

print(cname + ' predictions with default hyparameters for Test'+'\n')
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))  

print(cname + 'predictions with default hyparameters for Train'+'\n')
print('Accuracy')
print(accuracy_score(ytrain, tpred, normalize=True))

#compare bad predictions of different models; 
result=pd.concat([result, pd.DataFrame(pred)], axis=1) #to merge the predicted labels back to the dataset;
result.rename(columns={0:str(cname + '_pred')}, inplace=True)

LogisticRegression predictions with default hyparameters for Test

Accuracy
0.519666666667
[[113  17  29  42  44]
 [ 53  38  40 115  40]
 [ 14  17  78 258  77]
 [  8   4  37 642 337]
 [  3   3   3 300 688]]
             precision    recall  f1-score   support

          1       0.59      0.46      0.52       245
          2       0.48      0.13      0.21       286
          3       0.42      0.18      0.25       444
          4       0.47      0.62      0.54      1028
          5       0.58      0.69      0.63       997

avg / total       0.51      0.52      0.49      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.703


In [505]:
#########For RandomForest Regressor ONLY; 
###Stars 1, 2, 3 improves significantly, but stars 4 & 5, very hard to differentiate thus hard to adjust the threshold;
###Suggest to combine 4 & 5
'''
rtest=pd.DataFrame(ytest).reset_index(drop=True)
rltest=pd.concat([rtest, pd.DataFrame(pred)], axis=1) #result & labels; 
rltest.rename(columns={0:'reg_pred'}, inplace=True)
for i in [5, 4, 3, 2, 1]:
    print('star ' + str(i))
    print(rltest[rltest['stars']==i]['reg_pred'].describe())
#star 4 and 5 are very close, so as star 1 & 2; 
'''
#to define the regressor; 
def pred_gen(reg_value):
    if reg_value>4.3:
        return 5
    elif 3.8<reg_value<=4.3:
        return 4
    elif 3.0<reg_value<=3.8:
        return 3
    elif 2.4<reg_value<=3.0:
        return 2
    else:
        return 1


pred2=pd.DataFrame(pred)
pred2.rename(columns={0:'regpred'}, inplace=True)
pred2['predstar']=pred2['regpred'].apply(pred_gen)
pred2.head()

len(ytest)
ytst=ytest.reset_index(drop=True)
testres_all=pd.concat([ytst, pred2], axis=1)
testres_all.groupby(['stars', 'predstar']).count()

print('Accuracy')
print(accuracy_score(ytest, testres_all['predstar'], normalize=True))
print(confusion_matrix(ytest, testres_all['predstar']))
print(classification_report(ytest, testres_all['predstar'])) 

#testres_all[(testres_all['stars']==4) & (testres_all['predstar']!=4)]['regpred'].describe()

Accuracy
0.415333333333
[[114  45  54  19  13]
 [ 88  64  79  39  16]
 [ 32  73 155 137  47]
 [ 21  41 262 387 317]
 [ 13  28 132 298 526]]
             precision    recall  f1-score   support

          1       0.43      0.47      0.44       245
          2       0.25      0.22      0.24       286
          3       0.23      0.35      0.28       444
          4       0.44      0.38      0.41      1028
          5       0.57      0.53      0.55       997

avg / total       0.43      0.42      0.42      3000



In [430]:
#Sentiment analysis;
#dic of feature: coef pairs for star=1
for i in [3, 4, 2, 0, 1]:
    print('star: '+str(i+1))
    feature_to_coef = {word: coef for word, coef in zip(tfidfv.get_feature_names(), model.coef_[i, :])}   
        #i indicates which star, i=0,1,2,3 & star=1, 2, 3, 4
    keyfeas=[(k, v) for k, v in feature_to_coef.items() if 0.75<v<1]
    #print(keyfeas)
    print(len(keyfeas))

def sentiment_score(text):
    neg_words={'awkwardly', 'awkward', 'butt', 'discount', 'disappointed', 'disappoint', 
               'dismay', 'dumb', 'manner', 'sue', 'mean', 'meeting', 'monte', 'unfriendly', 'nasty', 'refuse', 
               'avoid', 'soggy', 'mistake', 'upset', 'filthy', 'sad', 'lousy', 'worst',  'unprofessional', 
               'rudely','unfortunately', 'screw', 'poorly', 'crap', 'crappy', 'tasteless', 'ignore', 'disaster',
               'overpriced', 'wrong', 'sick', 'inedible', 'stale', 'dirty', 'disappointment', 'sorry', 'yuck', 
               'flavorless', 'mess', 'dont', 'slow', 'lack', 'wouldnt', 'disappointed', 'terrible', 'overprice', 
               'gross', 'waste', 'poor', 'leave', 'disgust', 'awful', 'mediocre', 'suck', 'bland', 'wasnt', 
               'rude', 'didnt', 'horrible', 'bad'}
                   
    pos_words={'ambiance', 'ambience', 'awhile', 'delight', 'generous', 'hilarious', 'homey', 'hometown', 'home'
               'performance', 'performer', 'recover', 'tremendous', 'fantastically', 'fantastic', 'friendly', 'funny', 
               'goodwill', 'successfully', 'amaze', 'amazing', 'amazingly', 'magic', 'favourite', 'favorite', 'extraordinarily'
               'great', 'love', 'best', 'delicious', 'perfect', 'awesome', 'recommend', 'excellent', 'highly', 'fresh', 
               'friendly', 'wonderful', 'perfectly', 'like', 'beautiful', 'fabulous', 'cool', 'super', 'heaven', 'incredible'}   
    
    score=0
    for i in str(text).split():
        if i in pos_words:
            score+=1
        elif i in neg_words:
            score-=1
    return score
    
samp=data.loc[:1000, ['stars', 'textc']]
samp['sentiment']=samp['textc'].apply(sentiment_score)
print(samp.corr())  #to test the corr of star with sentiment, 0.44+, not bad.

data['sentiment']=data['textc'].apply(sentiment_score)
print(data.corr())  #sentiment now is 0.45 corr with stars; 

star: 4
87
star: 5
77
star: 3
55
star: 1
29
star: 2
63
              stars  sentiment
stars      1.000000   0.449947
sentiment  0.449947   1.000000


In [450]:
#to use sentiment and remove old features; 
neg_words={'awkwardly', 'awkward', 'butt', 'discount', 'disappointed', 'disappoint', 
           'dismay', 'dumb', 'manner', 'sue', 'mean', 'meeting', 'monte', 'unfriendly', 'nasty', 'refuse', 
           'avoid', 'soggy', 'mistake', 'upset', 'filthy', 'sad', 'lousy', 'worst',  'unprofessional', 
           'rudely','unfortunately', 'screw', 'poorly', 'crap', 'crappy', 'tasteless', 'ignore', 'disaster',
           'overpriced', 'wrong', 'sick', 'inedible', 'stale', 'dirty', 'disappointment', 'sorry', 'yuck', 
           'flavorless', 'mess', 'dont', 'slow', 'lack', 'wouldnt', 'disappointed', 'terrible', 'overprice', 
           'gross', 'waste', 'poor', 'leave', 'disgust', 'awful', 'mediocre', 'suck', 'bland', 'wasnt', 
           'rude', 'didnt', 'horrible', 'bad'}
pos_words={'ambiance', 'ambience', 'awhile', 'delight', 'generous', 'hilarious', 'homey', 'hometown', 'home',
           'performance', 'performer', 'recover', 'tremendous', 'fantastically', 'fantastic', 'friendly', 'funny', 
           'goodwill', 'successfully', 'amaze', 'amazing', 'amazingly', 'magic', 'favourite', 'favorite', 'extraordinarily',
           'great', 'love', 'best', 'delicious', 'perfect', 'awesome', 'recommend', 'excellent', 'highly', 'fresh', 
           'friendly', 'wonderful', 'perfectly', 'like', 'beautiful', 'fabulous', 'cool', 'super', 'heaven', 'incredible'}

###############to be ued for further word removal, while using sentiment###############
####TURN OUT: DO NOT REMOVE!!!
no_sentiment_feat=['feat_'+i for i in list(pos_words.union(neg_words))]  

In [154]:
'''
ty=pd.DataFrame(ytrain).reset_index(drop=True)
ty2=pd.concat([ty, trainfull], axis=1)
#trainfull, ytrain
ty2.corr()
'''
##Feature Selection: https://machinelearningmastery.com/feature-selection-machine-learning-python/ 
from sklearn.feature_selection import RFE
vs_rfe = RFE(LogisticRegression())
vs_rfe.fit(trainfull, ytrain)


'\nprint("Num Features: %d") % fit.n_features_\nprint("Selected Features: %s") % fit.support_\nprint("Feature Ranking: %s") % fit.ranking_\n'

In [207]:
#Feature enginnering 1: use RFE for half selected features (performance slightly worse than full set); 
allrfe=list(zip(trainfull, vs_rfe.support_, vs_rfe.ranking_))
yesfea=[i[0] for i in allrfe if i[1]==True] #list of supported features; 
nofea=[i[0] for i in allrfe if i[1]==False] #list of non-supported features; 

#combine trainfull (w/ tfidf) & label; 
labdf=pd.DataFrame(ytrain).reset_index(drop=True)
d1=trainfull.loc[:, yesfea] #2417 col; 
d2=trainfull.loc[:, nofea]  #2418 col;

df1=pd.concat([labdf, d1], axis=1)
df2=pd.concat([labdf, d2], axis=1)

df1corr=df1.corr().loc[:, ['stars']].sort_values(by='stars', ascending=False)
df2corr=df2.corr().loc[:, ['stars']].sort_values(by='stars', ascending=False)

print(df1corr.describe(percentiles=np.linspace(0, 1, 11))) #top and bottom 30% with CORR 0.01
print(df2corr.describe(percentiles=np.linspace(0, 1, 11))) #top and bottom 20% with CORR 0.01

#to use for model training; 
train_select=trainfull.loc[:, yesfea]
test_select=testfull.loc[:, yesfea]

In [239]:
#Feature engineering 2: use CORR(), and remove least relevant features (performance slightly worse than full set). 
train_label_full=pd.concat([pd.DataFrame(ytrain).reset_index(drop=True), trainfull], axis=1)
cormat=train_label_full.corr().loc[:, ['stars']].sort_values(by='stars', ascending=False)

print(cormat.describe(percentiles=[0.10, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
print(len([i for i in cormat['stars'] if abs(i)<0.004]))
cormat['features']=cormat.index

more_corr_feat=list(cormat[abs(cormat['stars'])>0.004]['features'])   #list for further removal of features
print(len(more_corr_feat))

#to use for model training; 
train_select=trainfull.loc[:, more_corr_feat]
train_select.drop('stars', axis=1, inplace=True)
test_select=testfull.loc[:, more_corr_feat]
test_select.drop('stars', axis=1, inplace=True)

In [449]:
#list(cormat[abs(cormat['stars'])>0.05]['features'])

In [306]:
#remove half features from trainfull/testfull
result=pd.DataFrame()
rs=300
global result

model=LogisticRegression(random_state=rs)
cname='LogisticRegression'

model.fit(train_select, ytrain)
pred=model.predict(test_select)
tpred=model.predict(train_select)
    
predproba=pd.DataFrame(model.predict_proba(test_select))
tpredproba=pd.DataFrame(model.predict_proba(train_select))

print(cname + ' predictions with default hyparameters for Test'+'\n')
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))  

print(cname + 'predictions with default hyparameters for Train'+'\n')
print('Accuracy')
print(accuracy_score(ytrain, tpred, normalize=True))

#compare bad predictions of different models; 
result=pd.concat([result, pd.DataFrame(pred)], axis=1) #to merge the predicted labels back to the dataset;
result.rename(columns={0:str(cname + '_pred')}, inplace=True)

LogisticRegression predictions with default hyparameters for Test

Accuracy
0.512666666667
[[ 98  22  20  53  52]
 [ 40  43  38 122  43]
 [  7  17  70 279  71]
 [  6   2  27 641 352]
 [  2   1   4 304 686]]
             precision    recall  f1-score   support

          1       0.64      0.40      0.49       245
          2       0.51      0.15      0.23       286
          3       0.44      0.16      0.23       444
          4       0.46      0.62      0.53      1028
          5       0.57      0.69      0.62       997

avg / total       0.51      0.51      0.48      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.677857142857


In [ ]:
'''
LogisticRegression predictions with default hyparameters for Test

Accuracy
0.514666666667
[[ 96  19  26  50  54]
 [ 40  40  41 121  44]
 [  8  12  80 266  78]
 [  5   2  32 644 345]
 [  2   2   4 305 684]]
             precision    recall  f1-score   support

          1       0.64      0.39      0.48       245
          2       0.53      0.14      0.22       286
          3       0.44      0.18      0.26       444
          4       0.46      0.63      0.53      1028
          5       0.57      0.69      0.62       997

avg / total       0.52      0.51      0.49      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.693857142857

#####XGBoost with all features
XGBoost predictions with default hyparameters for Test

Accuracy
0.479333333333
[[ 70   8  13  77  77]
 [ 21  19  30 159  57]
 [  6   6  49 296  87]
 [  3   0  22 649 354]
 [  5   1   6 334 651]]
             precision    recall  f1-score   support

          1       0.67      0.29      0.40       245
          2       0.56      0.07      0.12       286
          3       0.41      0.11      0.17       444
          4       0.43      0.63      0.51      1028
          5       0.53      0.65      0.59       997

avg / total       0.49      0.48      0.44      3000

XGBoostpredictions with default hyparameters for Train

Accuracy
0.601714285714

#####Logistic with R.F.E half selected features (slightly worse than full set, others highly similar);
LogisticRegression predictions with default hyparameters for Test

Accuracy
0.508666666667
[[ 98  17  25  48  57]
 [ 46  33  41 120  46]
 [  9  12  77 266  80]
 [  5   1  36 637 349]
 [  4   3   6 303 681]]
             precision    recall  f1-score   support

          1       0.60      0.40      0.48       245
          2       0.50      0.12      0.19       286
          3       0.42      0.17      0.24       444
          4       0.46      0.62      0.53      1028
          5       0.56      0.68      0.62       997

avg / total       0.50      0.51      0.48      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.678857142857


#Threshold adjustment: save for the last
#print(predproba)
#print(type(ytest))

a1=ytest.reset_index(drop=True)
a2=pd.DataFrame(pred, columns=['predicted'])

col = ['star_'+str(i) for i in range(1, 6)]
predproba.columns = col

a4=pd.concat([a1, a2, predproba], axis=1)
a4.head()

t5=a4[(a4['stars']==5) & (a4['predicted']==4)]
print(len(t5))
t5['star_5'].describe(percentiles=[0.01, 0.05, 0.10, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.75])
#Star_5 threshold can adjust to 0.3, can save half of "misclassified as star-4" to be 5
k=a4[(a4['star_5']<0.5) & (a4['star_5']>0.4) & (a4['star_4']>=0.5)]
k['star_4'].describe(percentiles=[0.01, 0.05, 0.10, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 
                                   0.5, 0.75])

#https://stackoverflow.com/questions/28716241/controlling-the-threshold-in-logistic-regression-in-scikit-learn
'''

In [ ]:
print('Conclusion')
print('Sentiment score is very useful to corrent the misclassification problem for stars 1')
print('RandomForest Regressor suggests very hard diffentiatio between stars 4 and 5 => combine both')